In [54]:
conn.rollback()

In [2]:
import psycopg2

In [3]:

# varolan veritabanına bağlan
conn = psycopg2.connect(
    dbname="postgres",
    user="postgres",
    password="123456",
    host="localhost"
)

# bir cursor oluşturun
cur = conn.cursor()


# park tablosunu oluşturun
cur.execute("""
    CREATE TABLE parks (
        id SERIAL PRIMARY KEY,
        name VARCHAR(255),
        area FLOAT,
        location GEOMETRY(POINT, 4326)
    );
""")

# değişiklikleri commit edin
conn.commit()



DuplicateTable: relation "parks" already exists


In [8]:
conn.rollback()

cur.execute("""
    ALTER TABLE parks
    ADD COLUMN city VARCHAR(255);
    ALTER TABLE parks
    ADD COLUMN boundaries GEOMETRY(POLYGON, 4326);
""")
conn.commit()

In [9]:
cur.execute("""
    INSERT INTO parks (name, area, location, boundaries)
    VALUES (
        'Central Park', 
        3.41, 
        ST_GeomFromText('POINT(-73.9712 40.7831)', 4326),
        ST_GeomFromText('POLYGON((-73.9814 40.7681, -73.9582 40.7681, -73.9582 40.7977, -73.9814 40.7977, -73.9814 40.7681))', 4326)
    );
""")

conn.commit()


In [10]:
# parkları çek
cur.execute("SELECT * FROM parks;")

# sonuçları al
rows = cur.fetchall()

# sonuçları ekrana bas
for row in rows:
    print(row)


(1, 'Central Park', 3.41, '0101000020E610000003780B24287E52C0B1BFEC9E3C644440', None, '0103000020E61000000100000005000000D712F241CF7E52C05F07CE1951624440BEC11726537D52C05F07CE1951624440BEC11726537D52C075029A081B664440D712F241CF7E52C075029A081B664440D712F241CF7E52C05F07CE1951624440')


In [12]:
cur.execute("""SELECT ST_AsText(location) FROM parks;""")
locationParks = cur.fetchall()
for loc in locationParks:
            print(loc)

('POINT(-73.9712 40.7831)',)


In [13]:
# Parks to insert
parks_to_insert = [
    ("Central Park", "POINT(-73.965355 40.782865)"),
    ("Golden Gate Park", "POINT(-123.4872 37.7694)"),
    ("Hyde Park", "POINT(-0.1657 51.5074)"),
    ("Ueno Park", "POINT(139.7713 35.7155)"),
]

# Insert parks into the table
for park in parks_to_insert:
    cur.execute("""
        INSERT INTO parks (name, location)
        VALUES (%s, ST_GeomFromText(%s, 4326));
    """, park)

# Commit changes
conn.commit()


In [14]:
# Define the point and distance
point = "POINT(-73.965355 40.782865)"  # Central Park
distance = 10000  # meters

# Find parks within the distance from the point
cur.execute("""
    SELECT name, ST_Distance(location::geography, ST_GeomFromText(%s, 4326)::geography)
    FROM parks
    WHERE ST_DWithin(location::geography, ST_GeomFromText(%s, 4326)::geography, %s);
""", (point, point, distance))

# Fetch and print results
rows = cur.fetchall()
for row in rows:
    print(row)


('Central Park', 494.06929829)
('Central Park', 0.0)


In [15]:
#conn.close()

In [10]:
# Define the point and distance
point = "POINT(-73.965355 40.782865)"  # Central Park
distance = 10000  # meters

# Find parks within the distance from the point
cur.execute("""
    SELECT name, ST_Distance(location::geography, ST_GeomFromText(%s, 4326)::geography)
    FROM parks
    WHERE ST_DWithin(location::geography, ST_GeomFromText(%s, 4326)::geography, %s);
""", (point, point, distance))

# Fetch and print results
rows = cur.fetchall()
for row in rows:
    print(row)

('Central Park', 0.0)


In [22]:
cur.execute("""
SELECT id, name, area, ST_AsText(location) as location, city, ST_AsText(boundaries) as boundaries FROM parks
""")

info = cur.fetchall()
for inf in info:
    print(inf)

(1, 'Central Park', 3.41, 'POINT(-73.9712 40.7831)', None, 'POLYGON((-73.9814 40.7681,-73.9582 40.7681,-73.9582 40.7977,-73.9814 40.7977,-73.9814 40.7681))')
(2, 'Central Park', None, 'POINT(-73.965355 40.782865)', None, None)
(3, 'Golden Gate Park', None, 'POINT(-123.4872 37.7694)', None, None)
(4, 'Hyde Park', None, 'POINT(-0.1657 51.5074)', None, None)
(5, 'Ueno Park', None, 'POINT(139.7713 35.7155)', None, None)
(6, 'Golden Gate Park', 2.23, 'POINT(-122.4862 37.7694)', None, 'POLYGON((-122.51059341430664 37.77100882052909,-122.51053619384766 37.769283121623616,-122.46183967590332 37.7694451567806,-122.46188259124756 37.7711708532109,-122.51059341430664 37.77100882052909))')
(7, 'Hyde Park', 7.83, 'POINT(-0.16573 51.507268)', None, 'POLYGON((-0.191875457763672 51.51167373609093,-0.191804885864258 51.50626141844558,-0.140314102172852 51.50639072960026,-0.14037036895752 51.51180303667296,-0.191875457763672 51.51167373609093))')
(8, 'Ueno Park', 9.6, 'POINT(139.771317 35.715476)', None

In [21]:
parks = [
    ('Golden Gate Park', 2.23, 'POINT(-122.4862 37.7694)', 'POLYGON((-122.51059341430664 37.77100882052909,-122.51053619384766 37.769283121623616,-122.46183967590332 37.7694451567806,-122.46188259124756 37.7711708532109,-122.51059341430664 37.77100882052909))'),
    ('Hyde Park',7.83, 'POINT(-0.165730 51.507268)', 'POLYGON((-0.19187545776367188 51.51167373609093,-0.1918048858642578 51.506261418445584,-0.14031410217285156 51.50639072960026,-0.14037036895751953 51.51180303667296,-0.19187545776367188 51.51167373609093))'),
    ('Ueno Park',9.6, 'POINT(139.771317 35.715476)', 'POLYGON((139.76801109313965 35.71924718861591,139.76795482635498 35.71281974412442,139.77562713623047 35.71268843440356,139.77569770812988 35.71911587908337,139.76801109313965 35.71924718861591))'),
]

for park in parks:
    cur.execute("""
        INSERT INTO parks (name, area, location, boundaries)
        VALUES (%s,%s, ST_GeomFromText(%s, 4326), ST_GeomFromText(%s, 4326));
    """, park)
    
conn.commit()


In [25]:
cur.execute("""
SELECT area, name from parks Where area >= 3
""")
area_greater_than_3 = cur.fetchall()
for area in area_greater_than_3:
    print(area)

(3.41, 'Central Park')
(7.83, 'Hyde Park')
(9.6, 'Ueno Park')


In [26]:
cur.execute("""
DELETE FROM parks AS p1
WHERE EXISTS (
SELECT *
FROM parks AS p2
WHERE p1.name = p2.name AND p1.id > p2.id
)
""")
cur.execute("""
SELECT name FROM parks 
""")
result = cur.fetchall()
conn.commit()
        


In [27]:
for res in result:
    print(res)

('Central Park',)
('Golden Gate Park',)
('Hyde Park',)
('Ueno Park',)


In [32]:
city_names=[("New York","Central Park"),("Washington", "Golden Gate Park"),("Houston","Hyde Park"),("Toronto","Ueno Park")]

for city, park in city_names:
    cur.execute(
    """
    UPDATE parks
    SET city = %s
    WHERE parks.name = %s
    """, (city, park))
conn.commit()


In [55]:

cur.execute("""
CREATE TABLE workers (
worker_id SERIAL PRIMARY KEY,
name VARCHAR(40),
salary FLOAT,
park_id INTEGER,
FOREIGN KEY (park_id) REFERENCES parks(id)
)
""")
conn.commit()

In [56]:
workers_data = [
    ("John Doe", 4000, 1),
    ("Jane Smith", 5000, 1),
    ("Mike Johnson", 4500, 3),
    ("Alice Williams", 6000, 4),
    ("Charlie Brown", 4000, 3),
    ("Maggie Davis", 5500, 3),
    ("Frank Miller", 4700, 4),
    ("Sophia Wilson", 5200, 4),
    ("Lucas Moore", 4800, 5),
    ("Olivia Taylor", 5300, 5)
]

for worker in workers_data:
    cur.execute(
        """
        INSERT INTO workers (name, salary, park_id)
        VALUES (%s, %s, %s)
        """, worker
    )

conn.commit()


In [57]:
# Inner Join
cur.execute("""
    SELECT parks.name AS park_name, workers.name AS worker_name
    FROM parks
    INNER JOIN workers ON parks.id = workers.park_id
""")

rows = cur.fetchall()
for row in rows:
    print(row)


('Central Park', 'John Doe')
('Central Park', 'Jane Smith')
('Golden Gate Park', 'Mike Johnson')
('Hyde Park', 'Alice Williams')
('Golden Gate Park', 'Charlie Brown')
('Golden Gate Park', 'Maggie Davis')
('Hyde Park', 'Frank Miller')
('Hyde Park', 'Sophia Wilson')
('Ueno Park', 'Lucas Moore')
('Ueno Park', 'Olivia Taylor')
